# Add time series to `System`

**Originally Contributed by**: Clayton Barrows

## Introduction

An example of how to parse add time series data to a `System` using [PowerSystems.jl](github.com/NREL-SIIP/PowerSystems.jl)

For example, a `System` created by [parsing a MATPOWER file](https://nbviewer.jupyter.org/github/NREL-SIIP/SIIPExamples.jl/blob/master/notebook/2_PowerSystems_examples/parse_matpower.ipynb)
doesn't contain any time series data. So a user may want to add time series to the `System`
### Dependencies
Let's use the 5-bus dataset we parsed in the MATPOWER example

In [1]:
using SIIPExamples
using PowerSystems
using JSON3

pkgpath = dirname(dirname(pathof(SIIPExamples)))
include(joinpath(pkgpath, "test", "2_PowerSystems_examples", "02_parse_matpower.jl"))

[ Info: extending matpower format with data: areas 1x3
[ Info: extending matpower format with data: gen_name 7x4
[ Info: extending matpower format by appending matrix "gen_name" in to "gen"
[ Info: reversing the orientation of branch 6 (4, 3) to be consistent with other parallel branches
[ Info: the voltage setpoint on generator 4 does not match the value at bus 4
[ Info: the voltage setpoint on generator 1 does not match the value at bus 1
[ Info: the voltage setpoint on generator 5 does not match the value at bus 10
[ Info: the voltage setpoint on generator 2 does not match the value at bus 1
[ Info: the voltage setpoint on generator 3 does not match the value at bus 3
[ Info: removing 1 cost terms from generator 4: [4000.0, 0.0]
[ Info: removing 1 cost terms from generator 1: [1400.0, 0.0]
[ Info: removing 1 cost terms from generator 5: [1000.0, 0.0]
[ Info: removing 1 cost terms from generator 2: [1500.0, 0.0]
[ Info: removing 3 cost terms from generator 6: Float64[]
[ Info: removi

System
======
System Units Base: SYSTEM_BASE
Base Power: 100.0
Base Frequency: 60.0

Components
==========
Num components: 30

9×3 DataFrame
 Row │ ConcreteType              SuperTypes                        
     │ String                    String                            
─────┼────────────────────────────────────────────────────────────────
   1 │ Arc                       Topology <: Component <: Infrast…
   2 │ Area                      AggregationTopology <: Topology …
   3 │ Bus                       Topology <: Component <: Infrast…
   4 │ Line                      ACBranch <: Branch <: Device <: …
   5 │ LoadZone                  AggregationTopology <: Topology …
   6 │ PhaseShiftingTransformer  ACBranch <: Branch <: Device <: …
   7 │ PowerLoad                 StaticLoad <: ElectricLoad <: St…
   8 │ RenewableDispatch         RenewableGen <: Generator <: Sta…
   9 │ ThermalStandard           ThermalGen <: Generator <: Stati…
                                                      1 column omitted

TimeSeriesContainer
===================
Components with time series data: 0
Total StaticTimeSeries: 0
Total Forecasts: 0

### Define pointers to time series files
For example, if we want to add a bunch of time series files, say one for each load and
one for each renewable generator, we need to define pointers to each .csv file containing
the time series in the following format (PowerSystems.jl also supports a CSV format for this file)

In [2]:
FORECASTS_DIR = joinpath(base_dir, "forecasts", "5bus_ts")
fname = joinpath(FORECASTS_DIR, "timeseries_pointers_da.json")
open(fname, "r") do f
    JSON3.@pretty JSON3.read(f)
end

[
  {
                       "component_name": "SolarBusC",
                 "normalization_factor": 1,
                                 "name": "max_active_power",
     "scaling_factor_multiplier_module": "PowerSystems",
                            "data_file": "./gen/Renewable/PV/da_solar5.csv",
                           "resolution": 3600,
                               "module": "PowerSystems",
                             "category": "Generator",
            "scaling_factor_multiplier": "get_max_active_power",
                           "simulation": "DAY_AHEAD",
                                 "type": "SingleTimeSeries"
  },
  {
                       "component_name": "WindBusA",
                 "normalization_factor": 1,
                                 "name": "max_active_power",
     "scaling_factor_multiplier_module": "PowerSystems",
                            "data_file": "./gen/Renewable/WIND/da_wind5.csv",
                           "resolution": 3600,
               

### Read and assign time series to `System` using these parameters.

In [3]:
add_time_series!(sys, fname)
sys

System
======
System Units Base: SYSTEM_BASE
Base Power: 100.0
Base Frequency: 60.0

Components
==========
Num components: 30

9×3 DataFrame
 Row │ ConcreteType              SuperTypes                        
     │ String                    String                            
─────┼────────────────────────────────────────────────────────────────
   1 │ Arc                       Topology <: Component <: Infrast…
   2 │ Area                      AggregationTopology <: Topology …
   3 │ Bus                       Topology <: Component <: Infrast…
   4 │ Line                      ACBranch <: Branch <: Device <: …
   5 │ LoadZone                  AggregationTopology <: Topology …
   6 │ PhaseShiftingTransformer  ACBranch <: Branch <: Device <: …
   7 │ PowerLoad                 StaticLoad <: ElectricLoad <: St…
   8 │ RenewableDispatch         RenewableGen <: Generator <: Sta…
   9 │ ThermalStandard           ThermalGen <: Generator <: Stati…
                                                      1 column omitted

TimeSeriesContainer
===================
Components with time series data: 5
Total StaticTimeSeries: 5
Total Forecasts: 0
Resolution: 60 minutes

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*